# Predicting Remaining Useful Life (advanced)
<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
    <img style="display:inline" width=15% src="https://upload.wikimedia.org/wikipedia/commons/e/e5/NASA_logo.svg" alt="NASA" />
</p>

This notebook has a more advanced workflow than [the other notebook](Simple%20Featuretools%20RUL%20Demo.ipynb) for predicting Remaining Useful Life (RUL). If you are a new to either this dataset or Featuretools, I would recommend reading the other notebook first. 

## Highlights
* Demonstrate how novel entityset structures improve predictive accuracy
* Build custom primitives using time-series functions from [tsfresh](https://github.com/blue-yonder/tsfresh)
* Improve Mean Absolute Error by tuning hyper parameters with [BTB](https://github.com/HDI-Project/BTB)

Here is a collection of mean absolute errors from both notebooks. Though we've used averages where possible (denoted by \*), the randomness in the Random Forest Regressor and how we choose labels from the train data changes the score.

|                                 | Train/Validation MAE|  Test MAE|
|---------------------------------|--------------------------------|
| Median Baseline                 | 72.06*              | 50.66*   |
| Simple Featuretools             | 40.92*              | 39.56    |
| Advanced: Custom Primitives     | 35.90*              | 28.84    |
| Advanced: Hyperparameter Tuning | 34.80*              | 27.85    |


# Step 1: Load Data
We load in the train data using the same function we used in the previous notebook:

In [1]:
import numpy as np
import pandas as pd
import featuretools as ft
import utils

data_path = 'data/train_FD004.txt'
data = utils.load_data(data_path)

data.head()

Loaded data with:
61249 Recordings
249 Engines
21 Sensor Measurements
3 Operational Settings


,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,index,time
index,,,,,,,,,,,,,,,,,,,,,
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,0,2000-01-01 00:00:00
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,1,2000-01-01 00:10:00
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,2,2000-01-01 00:20:00
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,3,2000-01-01 00:30:00
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,4,2000-01-01 00:40:00


We also make cutoff times by selecting a random cutoff time from the life of each engine. We're going to make 5 sets of cutoff times to use for cross validation.

In [2]:
from tqdm import tqdm

splits = 5
cutoff_time_list = []

for i in tqdm(range(splits)):
    cutoff_time_list.append(utils.make_cutoff_times(data))

cutoff_time_list[0].head()

100%|██████████| 5/5 [00:03<00:00,  1.51it/s]


,engine_no,cutoff_time,RUL
index,,,
1,1,2000-01-02 21:40:00,46
2,2,2000-01-04 00:20:00,185
3,3,2000-01-06 06:10:00,169
4,4,2000-01-07 20:50:00,211
5,5,2000-01-09 15:50:00,146


We're going to do something fancy for our entityset. The values for `operational_setting` 1-3 are continuous but create an implicit relation between different engines. If two engines have a similar `operational_setting`, it could indicate that we should expect the sensor measurements to mean similar things. We make clusters of those settings using [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) from scikit-learn and make a new entity from the clusters.

In [3]:
from sklearn.cluster import KMeans

nclusters = 50

def make_entityset(data, nclusters, kmeans=None):
    X = data[['operational_setting_1', 'operational_setting_2', 'operational_setting_3']]
    if kmeans:
        kmeans=kmeans
    else:
        kmeans = KMeans(n_clusters=nclusters).fit(X)
    data['settings_clusters'] = kmeans.predict(X)
    
    es = ft.EntitySet('Dataset')
    es.entity_from_dataframe(dataframe=data,
                             entity_id='recordings',
                             index='index',
                             time_index='time')

    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='engines',
                        index='engine_no')
    
    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='settings_clusters',
                        index='settings_clusters')
    
    return es, kmeans
es, kmeans = make_entityset(data, nclusters)
es

Entityset: Dataset
  Entities:
    recordings [Rows: 61249, Columns: 29]
    settings_clusters [Rows: 50, Columns: 2]
    engines [Rows: 249, Columns: 2]
  Relationships:
    recordings.engine_no -> engines.engine_no
    recordings.settings_clusters -> settings_clusters.settings_clusters

# Step 2: DFS and Creating a Model
In addition to changing our `EntitySet` structure, we're also going to use the [Complexity](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.cid_ce) time series primitive from the package [tsfresh](https://github.com/blue-yonder/tsfresh). Any function that takes in a pandas `Series` and outputs a float can be converted into an aggregation primitive using the `make_agg_primitive` function as shown below.

In [4]:
from featuretools.primitives import make_agg_primitive
import featuretools.variable_types as vtypes

from tsfresh.feature_extraction.feature_calculators import (number_peaks, mean_abs_change, 
                                                            cid_ce, last_location_of_maximum, length)


Complexity = make_agg_primitive(lambda x: cid_ce(x, False),
                              input_types=[vtypes.Numeric],
                              return_type=vtypes.Numeric,
                              name="complexity")

fm, features = ft.dfs(entityset=es, 
                      target_entity='engines',
                      agg_primitives=['last', 'max', Complexity],
                      trans_primitives=[],
                      chunk_size=.26,
                      cutoff_time=cutoff_time_list[0],
                      max_depth=3,
                      verbose=True)

fm.to_csv('advanced_fm.csv')
fm.head()

Built 302 features
Elapsed: 04:34 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 4/4 chunks


,COMPLEXITY(recordings.sensor_measurement_15),MAX(recordings.sensor_measurement_18),COMPLEXITY(recordings.sensor_measurement_10),MAX(recordings.sensor_measurement_14),MAX(recordings.sensor_measurement_6),LAST(recordings.sensor_measurement_12),LAST(recordings.sensor_measurement_19),COMPLEXITY(recordings.operational_setting_3),COMPLEXITY(recordings.time_in_cycles),LAST(recordings.sensor_measurement_16),...,MAX(recordings.settings_clusters.COMPLEXITY(recordings.sensor_measurement_17)),LAST(recordings.settings_clusters.COMPLEXITY(recordings.operational_setting_1)),MAX(recordings.settings_clusters.COMPLEXITY(recordings.sensor_measurement_15)),COMPLEXITY(recordings.settings_clusters.COMPLEXITY(recordings.sensor_measurement_7)),MAX(recordings.settings_clusters.COMPLEXITY(recordings.sensor_measurement_5)),LAST(recordings.settings_clusters.COMPLEXITY(recordings.sensor_measurement_6)),COMPLEXITY(recordings.settings_clusters.COMPLEXITY(recordings.sensor_measurement_12)),COMPLEXITY(recordings.settings_clusters.MAX(recordings.sensor_measurement_3)),LAST(recordings.settings_clusters.LAST(recordings.operational_setting_1)),RUL
engine_no,,,,,,,,,,,,,,,,,,,,,
1,18.040362,2388,2.930785,8157.56,21.59,374.73,100.00,353.270435,16.552945,0.03,...,6.928203,0.002175,0.148429,25.370561,0.0,0.000000,27.399104,2433.211812,10.0064,46
2,11.008591,2388,1.824226,8138.46,21.61,315.16,100.00,211.660105,10.630146,0.02,...,8.944272,0.000877,0.306435,59.896197,0.0,0.040000,60.370021,1526.512050,20.0075,185
3,14.168354,2388,2.347637,8147.30,21.60,164.92,84.93,268.328157,11.704700,0.02,...,11.045361,0.002657,0.371621,72.712350,0.0,0.043589,73.808480,1919.713760,24.9990,169
4,8.893902,2388,1.432934,8146.95,21.61,164.46,84.93,174.355958,7.874008,0.02,...,12.727922,0.003161,0.471283,62.187586,0.0,0.041231,61.857775,1333.727758,25.0074,211
5,7.182040,2388,1.285535,8126.22,21.61,314.57,100.00,126.491106,6.782330,0.02,...,13.266499,0.002480,0.472573,49.409679,0.0,0.062450,50.337343,1054.184634,19.9992,146


We build 4 more feature matrices with the same feature set but different cutoff times. That lets us test the pipeline multiple times before using it on test data.

In [5]:
fm_list = [fm]
for i in tqdm(range(1, splits)):
    fm = ft.calculate_feature_matrix(entityset=make_entityset(data, nclusters, kmeans=kmeans)[0], 
                                     features=features, 
                                     chunk_size=.26, 
                                     cutoff_time=cutoff_time_list[i])
    fm_list.append(fm)

100%|██████████| 4/4 [15:20<00:00, 236.21s/it]


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE
def pipeline_for_test(fm_list, hyperparams={'n_estimators':100, 'max_feats':50, 'nfeats':50}, do_selection=False):
    scores = []
    regs = []
    selectors = []
    for fm in fm_list:
        X = fm.copy().fillna(0)
        y = X.pop('RUL')
        reg = RandomForestRegressor(n_estimators=int(hyperparams['n_estimators']), 
                                    max_features=min(int(hyperparams['max_feats']), 
                                                     int(hyperparams['nfeats'])))
        X_train, X_test, y_train, y_test = train_test_split(X, y)
        if do_selection:
            reg2 = RandomForestRegressor(n_jobs=3)
            selector = RFE(reg2, int(hyperparams['nfeats']), step=25)
            selector.fit(X_train, y_train)
            X_train = selector.transform(X_train)
            X_test = selector.transform(X_test)
            selectors.append(selector)
        reg.fit(X_train, y_train)
        regs.append(reg)
        
        preds = reg.predict(X_test)
        scores.append(mean_absolute_error(preds, y_test))
    return scores, regs, selectors    
scores, regs, selectors = pipeline_for_test(fm_list)
print([float('{:.1f}'.format(score)) for score in scores])
print('Average MAE: {:.1f}, Std: {:.2f}\n'.format(np.mean(scores), np.std(scores)))

most_imp_feats = utils.feature_importances(fm_list[0], regs[0])

[42.0, 33.6, 32.5, 37.0, 39.9]
Average MAE: 37.0, Std: 3.60

1: COMPLEXITY(recordings.settings_clusters.LAST(recordings.sensor_measurement_13)) [0.060]
2: COMPLEXITY(recordings.settings_clusters.MAX(recordings.sensor_measurement_13)) [0.054]
3: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_13)) [0.033]
4: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_11)) [0.028]
5: MAX(recordings.sensor_measurement_13) [0.027]
-----



In [7]:
data_test = utils.load_data('data/test_FD004.txt')

es_test, _ = make_entityset(data_test, nclusters, kmeans=kmeans)
fm_test = ft.calculate_feature_matrix(entityset=es_test, features=features, verbose=True, chunk_size='cutoff time')
X = fm_test.copy().fillna(0)
y = pd.read_csv('data/RUL_FD004.txt', sep=' ', header=-1, names=['RUL'], index_col=False)
preds = regs[0].predict(X)
print('Mean Abs Error: {:.2f}'.format(mean_absolute_error(preds, y)))

Loaded data with:
41214 Recordings
248 Engines
21 Sensor Measurements
3 Operational Settings
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
Mean Abs Error: 34.19


# Step 3: Feature Selection and Scoring
Here, we'll use [Recursive Feature Elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html). In order to set ourselves up for later optimization, we're going to write a generic `pipeline` function which takes in a set of hyperparameters and returns a score. Our pipeline will first run `RFE` and then split the remaining data for scoring by a `RandomForestRegressor`. We're going to pass in a list of hyperparameters, which we will tune later. 

Lastly, we can use that selector and regressor to score the test values.

# Step 4: Hyperparameter Tuning
Because of the way we set up our pipeline, we can use a Gaussian Process to tune the hyperparameters. We will use [BTB](https://github.com/HDI-Project/BTB) from the [HDI Project](https://github.com/HDI-Project). This will search through the hyperparameters `n_estimators` and `max_feats` for RandomForest, and the number of features for RFE to find the hyperparameter set that has the best average score.

In [8]:
from btb import HyperParameter, ParamTypes
from btb.tuning import GP

def run_btb(fm_list, n=30):
    hyperparam_ranges = [
            ('n_estimators', HyperParameter(ParamTypes.INT, [10, 200])),
            ('max_feats', HyperParameter(ParamTypes.INT, [5, 50])),
            ('nfeats', HyperParameter(ParamTypes.INT, [10, 70])),
    ]
    tuner = GP(hyperparam_ranges)

    tested_parameters = np.zeros((n, len(hyperparam_ranges)), dtype=object)
    scores = []
    
    print('[n_est, max_feats, nfeats]')
    best = 45

    for i in tqdm(range(n)):
        hyperparams = tuner.propose()
        cvscores, regs, selectors = pipeline_for_test(fm_list, hyperparams=hyperparams, do_selection=True)
        bound = np.mean(cvscores)
        tested_parameters[i, :] = hyperparams
        tuner.add(hyperparams, -np.mean(cvscores))
        if np.mean(cvscores) + np.std(cvscores) < best:
            best = np.mean(cvscores)
            best_hyperparams = hyperparams
            best_reg = regs[0]
            best_sel = selectors[0]
            print('{}. {} -- Average MAE: {:.1f}, Std: {:.2f}'.format(i, 
                                                                      best_hyperparams, 
                                                                      np.mean(cvscores), 
                                                                      np.std(cvscores)))
            print('Raw: {}'.format([float('{:.1f}'.format(s)) for s in cvscores]))

    return best_hyperparams, (best_sel, best_reg)

best_hyperparams, best_pipeline = run_btb(fm_list, n=30)


[n_est, max_feats, nfeats]

  0%|          | 0/30 [00:00<?, ?it/s]

/Users/lecrintech/projects/python/demos-featuretools/demo4/predict-remaining-useful-life/env2.7/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
  3%|▎         | 1/30 [00:23<11:18, 23.41s/it]

0. {'n_estimators': 177, 'max_feats': 48, 'nfeats': 42} -- Average MAE: 37.8, Std: 2.52
Raw: [42.6, 37.2, 37.4, 35.9, 35.6]


 80%|████████  | 24/30 [07:30<01:53, 18.92s/it]

23. {'n_estimators': 29, 'max_feats': 42, 'nfeats': 42} -- Average MAE: 35.0, Std: 1.54
Raw: [35.3, 33.3, 36.3, 33.1, 36.9]


100%|██████████| 30/30 [09:25<00:00, 18.56s/it]


In [9]:
X = fm_test.copy().fillna(0)
y = pd.read_csv('data/RUL_FD004.txt', sep=' ', header=-1, names=['RUL'], index_col=False)

preds = best_pipeline[1].predict(best_pipeline[0].transform(X))
score = mean_absolute_error(preds, y)
print('Mean Abs Error on Test: {:.2f}'.format(score))
most_imp_feats = utils.feature_importances(X.iloc[:, best_pipeline[0].support_], best_pipeline[1])

Mean Abs Error on Test: 35.33
1: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_4)) [0.107]
2: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_13)) [0.106]
3: COMPLEXITY(recordings.settings_clusters.LAST(recordings.sensor_measurement_13)) [0.084]
4: MAX(recordings.time_in_cycles) [0.083]
5: MAX(recordings.sensor_measurement_13) [0.066]
-----



# Appendix: Averaging old scores
To make a fair comparison between the previous notebook and this one, we should average scores where possible. The work in this section is exactly the work in the previous notebook plus some code for taking the average in the validation step.

In [10]:
from featuretools.primitives import Min
old_fm, features = ft.dfs(entityset=es, 
                      target_entity='engines',
                      agg_primitives=['last', 'max', 'min'],
                      trans_primitives=[],
                      cutoff_time=cutoff_time_list[0],
                      max_depth=3,
                      verbose=True)

old_fm_list = [old_fm]
for i in tqdm(range(1, splits)):
    old_fm = ft.calculate_feature_matrix(entityset=make_entityset(data, nclusters, kmeans=kmeans)[0], 
                                     features=features, 
                                     cutoff_time=cutoff_time_list[i])
    old_fm_list.append(fm)

old_scores = []
median_scores = []
for fm in old_fm_list:
    X = fm.copy().fillna(0)
    y = X.pop('RUL')
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    reg = RandomForestRegressor()
    reg.fit(X_train, y_train)
    preds = reg.predict(X_test)
    old_scores.append(mean_absolute_error(preds, y_test))
    
    medianpredict = [np.median(y_train) for _ in y_test]
    median_scores.append(mean_absolute_error(medianpredict, y_test))

print([float('{:.1f}'.format(score)) for score in old_scores])
print('Average MAE: {:.2f}, Std: {:.2f}\n'.format(np.mean(old_scores), np.std(old_scores)))

print([float('{:.1f}'.format(score)) for score in median_scores])
print('Baseline by Median MAE: {:.2f}, Std: {:.2f}\n'.format(np.mean(median_scores), np.std(median_scores)))

Built 302 features
Elapsed: 05:36 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


100%|██████████| 4/4 [19:15<00:00, 292.04s/it]


[45.5, 42.2, 46.9, 42.4, 43.2]
Average MAE: 44.04, Std: 1.85

[71.0, 66.6, 93.3, 76.1, 81.0]
Baseline by Median MAE: 77.62, Std: 9.22



In [11]:
y = pd.read_csv('data/RUL_FD004.txt', sep=' ', header=-1, names=['RUL'], index_col=False)
median_scores_2 = []
for ct in cutoff_time_list:
    medianpredict2 = [np.median(ct['RUL'].values) for _ in y.values]
    median_scores_2.append(mean_absolute_error(medianpredict2, y))
print([float('{:.1f}'.format(score)) for score in median_scores_2])
print('Baseline by Median MAE: {:.2f}, Std: {:.2f}\n'.format(np.mean(median_scores_2), np.std(median_scores_2)))

[47.7, 49.0, 49.8, 48.1, 49.2]
Baseline by Median MAE: 48.75, Std: 0.76



In [12]:
# Save output files

import os

try:
    os.mkdir("output")
except:
    pass

fm.to_csv('output/advanced_train_feature_matrix.csv')
cutoff_time_list[0].to_csv('output/advanced_train_label_times.csv')
fm_test.to_csv('output/advanced_test_feature_matrix.csv')

<p>
    <img src="https://www.featurelabs.com/wp-content/uploads/2017/12/logo.png" alt="Featuretools" />
</p>

Featuretools was created by the developers at [Feature Labs](https://www.featurelabs.com/). If building impactful data science pipelines is important to you or your business, please [get in touch](https://www.featurelabs.com/contact.html).